In [1]:
import pandas as pd
import numpy as np
from loading import *

## Loading Data

- tradition distances integer values between 0 and n
- transformation distances integer values between 1 and n
- jump tradition and transformation distances for thos publications without links in either of them

For example, if a publication has Nan in both tradition and jump_tradition, it is completely disconnected.
Nans basically mean that there is no path to the current year (the publication according to our model *cannot* be alive).

In [2]:
year = 1990
trans_dists = load_transformation_distances(year)
trad_dists = load_tradition_distances(year)
jump_trad_dists = load_jump_tradition_distances(year)
jump_trans_dists = load_jump_transformation_distances(year)

See that the corresponding claim holds, only the concept distances contain all the publications:

In [3]:
len(trans_dists), len(trad_dists), len(jump_trans_dists), len(jump_trad_dists)

(44603, 75153, 56392, 25842)

In [4]:
len(trans_dists) + len(jump_trans_dists), len(trad_dists) + len(jump_trad_dists)

(100995, 100995)

We can see that in this case all the publications would be connected to 1990 by **both** tradition and transformation. The topic jump accomplishes this in the corresponding year.

For convenience, simply load the whole table for a given year:

In [5]:
merged_dists = load_distances(year)
merged_dists.head()

,transformation_distance,tradition_distance,jump_tradition_distance,jump_transformation_distance
pub_id,,,,
00ca027b-5174-40fa-bf63-9a97c2a5f518,NaN,0.0,NaN,0.0
02804a61-a180-4f77-8edf-ff630ddd5ceb,2.0,0.0,NaN,NaN
02a186be-84bc-4c44-9fd6-2b15d9123607,NaN,NaN,0.0,0.0
02d48327-6f23-4906-b2fb-c1ff66bf6b74,2.0,NaN,0.0,NaN
03cda805-9746-48bb-a04d-02c2dac201c7,3.0,1.0,NaN,NaN


It might further be interesting to remove the unconnected components before conducting the analysis. We have to see how it affects the analysis though. Apparently, using the topic jump allows us to fit nearly every paper.

In [6]:
pubs = load_disconnected_publications()
filtered_merged_dists = merged_dists.loc[merged_dists.index.difference(pubs.index)]

In [7]:
len(merged_dists), len(filtered_merged_dists)

(100995, 96487)

## Data Analysis